<a href="https://colab.research.google.com/github/boom-bam-boop/Basic-GAN/blob/main/Copy_of_Yet_another_copy_of_Court_agent_ex_original_submission_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:

from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient
import re

In [117]:
import openai
from openai import OpenAI


In [118]:
global_history: List[Dict[str, str]] = []
global summ
summ = ""

In [119]:
class Witness:


    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = ""):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = OpenAI(
               api_key="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",
                base_url="https://api.groq.com/openai/v1"
          )          # make sure this env‑var is set


    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        """prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt"""

        return messages

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        global summ
        prompt = self._format_prompt(user_msg)
        """completion = self.client.text_generation(
            prompt,
            max_new_tokens=250,
            temperature=0.5,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )"""
        response = self.client.chat.completions.create(
        model="llama3-70b-8192",
        messages=prompt,
        temperature=0.3,
        max_tokens=150
        )

        answer = (response.choices[0].message.content).strip()
        # keep chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        summary = summarizer.client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=prompt,
        temperature=0.3,
        max_tokens=63
        )
        summ += summary.choices[0].message.content
        return answer


In [120]:

# Two–Lawyer Agents (Defense & Prosecution)




class LawyerAgent:


    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = ""):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = OpenAI(
               api_key="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",
                base_url="https://api.groq.com/openai/v1"          # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.


        return messages

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        global summ
        prompt = self._format_prompt(user_msg)

        response = self.client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=prompt,
        temperature=0.3,
        max_tokens=150
        )
        answer = (response.choices[0].message.content).strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        summary = summarizer.client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=prompt,
        temperature=0.3,
        max_tokens=63
        )
        summ += summary.choices[0].message.content

        return answer




# System prompts

DEFENSE_SYSTEM = """
You are **Raj Malani**, lead *defense counsel*, talk as if you are an actual lawyer in court, and talking to actual people.
Goals:
be concise, and to the point
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Witness Calling :
Call a witness everytime as it strengthens your case
You can create witness names and their occupations to support your trial, if you want to call witness put _WITNESS_ in your message.
Do not add any follow up or summary.
End your statement by questioning the witness, do not say anything further like thanking the witness, or saying anything on behalf of the witness.

Style:
Be brief and to the point, but accurate and precise in your points, do not waste your words
Simple english
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
DO not leave any sentence incomplete, you must end with a complete sentence
If you wish to conclude your case, say _CONCLUDE_,only conclude when both the prosecution and defense have finished with their points and there is some clear conclusion.
"""

PROSECUTION_SYSTEM = """
You are **Aryaman Tiwari**, *Assistant District Attorney* for the State, talk as if you are an actual lawyer in court, and talking to actual people.
Goals:
be concise, and to the point
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.

Witness Calling :
Call witnesses everytime as it strengthens your case
You can create witness names and their occupations to support your trial, if you want to call witness put _WITNESS_.
Do not add any follow up or summary.
End your statement by questioning the witness, do not say anything further like thanking the witness, or saying anything on behalf of the witness.



Style:
Be brief and to the point, but accurate and precise in your points, do not waste your words
• Formal but plain, and simple English; persuasive, with confident tone.
Ethics:
• Duty is to win, even if unethically. Do not concede points.
DO not leave any sentence incomplete, you must end with a complete sentence
If you wish to conclude your case, say _CONCLUDE_,only conclude when both the prosecution and defense have finished with their points and there is some clear conclusion.
"""
JUDGE_SYSTEM = """
You are **Judge Chandrachud**, presiding over a criminal trial.
Responsibilities:
Say as dew words as possible
• Ensure a fair and orderly courtroom.
• Rule on objections and rulings.
• Deliver the verdict based on the evidence presented.
It is not your job to fight the case, call witnesses, or anything else.
You give only 0 or 1 as output, 0 if you think defendant is innocent, and 1 if you thing defendant is guilty.
Be brief and to the point, but accurate and precise in your points, do not waste your words
"""
DEFENDANT_SYSTEM = """
You are John Doe, the defendant in a criminal trial. You will be questioned by the prosecution and defense attorneys. You will be asked to answer questions about the case. You want to prove your innocence, and are willing to lie for it."""

PLAINTIFF_SYSTEM = """
You are the Employer, the plaintiff in a criminal trial. You will be questioned by the prosecution and defense attorneys. You will be asked to answer questions about the case. You want to prove the defendant guilty, and are willing to lie for it."""

WITNESS_SYSTEM = """You are a witness in a court of law. You must answer all questions truthfully and to the best of your ability.
You must not speculate or offer opinions unless specifically asked to do so. If you do not know the answer to a question, you should say "I don't know" or "I don't recall.
" You should avoid using slang or jargon. You should address the judge as "Your Honor" and the attorneys as "Mr." or "Ms." followed by their last names.
You are a neutral party and should not express bias towards either the prosecution or the defense. You should only provide information that is relevant to the case.
You should not discuss the case with anyone outside of the courtroom.
Do not give your own opinions or thoughts on the case, or about who is guilty or not guilty, simply answer the questions asked of you
Be brief and to the point, but accurate and precise in your answers, do not waste your words
Sometimes even if you think the defendant is guilty, make teh verdict not guilty, giving output 0"""

# the two agents

defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
judge = LawyerAgent("Judge", JUDGE_SYSTEM)
summarizer = LawyerAgent("Summarizer", "You are a summarizer, you summarize the text given to you in a concise manner, in as few words as possible")









In [121]:
import pandas as pd

In [122]:

df = pd.read_csv("/content/cases.csv", nrows = 50)

In [123]:
confidence_threshold = 0.8

In [124]:
import time

In [125]:
def verdict(case_background):
  prosecution.history = []
  defense.history = []
  judge.history = []

  print("Judge : ", judge.respond(f"Very short opening statement to the court as a judge. Background: {case_background}"))
  x = 0
  while(x < 5):
        x+=1
        # Prosecutor goes first
        answer = prosecution.respond(
        f"Put forward your points and statements using evidence and information to prove the defendant guilty, take into account everything that has been said till now by both the witnesses and the defense, call any other witnesses if needed. Background: {case_background}"
        )
        time.sleep(2)
        print("Prosecution : ", answer, "\n\n\n")
        if "_CONCLUDE_" in answer:
            break;
        elif "_WITNESS_" in answer:
            witness_name = answer.split("_WITNESS_")[1].strip()
            witness1 = Witness(witness_name,WITNESS_SYSTEM)
            wit = witness1.respond(f"Respond to questions asked by the prosecutor, do not lie or fabricate, answer carefully and truthfully. Prosecutor's questions : {answer}")
            time.sleep(2)
            print("Witness : ", wit, "\n\n\n")



        #Defense respond
        answerd = defense.respond(
        f"Put forward your points and statements using evidence and information to prove your client as innocent, take into account everything that has been said till now by both the witnesses and the prosecution, call any other witnesses if needed.Background: {case_background}"
        )
        time.sleep(2)
        print("Defense : ", answerd, "\n\n\n")
        if "_CONCLUDE_" in answerd:
            break;
        elif "_WITNESS_" in answerd:
            witness_name = answerd.split("_WITNESS_")[1].strip()
            witness2 = Witness(witness_name,WITNESS_SYSTEM)
            wit2 = witness2.respond(f"Respond to questions asked by the defense, do not lie or fabricate, answer carefully and truthfully.  Defense's questions : {answerd}")
            time.sleep(2)
            print("Witness : ", wit2, "\n\n\n")



  verdict = judge.respond(f"Give your final decision after taking into account the statements of the prosecution, defense, and the witnesses, output 1 for defense being proven guilty, 0 for defense being proven innocent. Summary of the case : {summ}")
  print("FINAL VERDICT : ", verdict, "\n")
  verdict_final = "1" in verdict
  return verdict_final






In [ ]:
import multiprocessing
def process_case(case_data):
    # Unpack the case data
    index, case_text = case_data
    max_length = 4000
    case_text = case_text[:max_length]
    year = df.loc[index, 'id']

    # Split case_text into chunks of 1000 characters
    chunk_size = 1000  # Adjust as needed
    chunks = [case_text[i:i + chunk_size] for i in range(0, len(case_text), chunk_size)]

    # Summarize each chunk
    summary = ""
    for chunk in chunks:
        summary += summarizer.respond(f"Summarize the following text: {chunk}")

    case_background = summary

    # Get the verdict
    case_verdict = verdict(case_background)  # This function remains the same

    return [year, case_verdict]

def run_normal_processing(df):
    results = []
    for index, row in df.iterrows():
        case_data = [index, row['text']]
        result = process_case(case_data)
        results.append(result)

    return results
results = run_normal_processing(df)

Judge :  "This court is now in session. The case before us is a civil appeal and SLP challenging the Central Administrative Tribunal's power to review a penalty imposed on a government servant, specifically Parmanand, a timekeeper. I will ensure a fair and orderly trial. Let the proceedings begin."
Prosecution :  Your Honor, the prosecution's case against Parmanand, the accused, is clear-cut and supported by overwhelming evidence. The accused, a timekeeper, was entrusted with the responsibility of maintaining attendance records and issuing identity cards. However, he abused his position by preparing a fake pay roll and identity card to facilitate Suraj Singh to impersonate Ashok Kumar and withdraw a sum of Rs.238-90.

Exhibit A-1, the attendance record, reveals that Parmanand falsely marked Ashok Kumar's attendance, while Exhibit A-2, the identity card, was created to aid Suraj Singh in his impersonation. The investigation revealed that Parmanand's actions were not only a breach of his

In [ ]:
with open('verdicts.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'verdict'])  # Write header
    writer.writerows(results)  # Write data rows